In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
#import input_data
import tensor_mnist.mnist as input_data

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST ready")

filepath:data/train-images-idx3-ubyte.gz
Extracting data/train-images-idx3-ubyte.gz
filepath:data/train-labels-idx1-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
filepath:data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
filepath:data/t10k-labels-idx1-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST ready


* data:n*784  
* conv1:卷积核大小3*3*1，卷积核个数64，stride=1*1
* pooling1:2*2 ,stride=2*2
* conv2:卷积核大小3*3*64(由上一层输入的64个特征图)，卷积核个数128  
* pooling2：2*2,stride=2*2  

* 全连接层1：1024，全连接层2（分类结果）：10

* 输入是28*28大小，计算特征图大小：[h|w-filer(h|w)+2*padding]/stride + 1
(28-3+2)/1+1=28,说明卷积层没有改变图像的大小。  

* pooling层使图像变小了，2*2变为1*1，h和w都变为原来的1/2，即14*14 
* 所以经过第一层pooling后得到的是14*14，  
* 第二层pooling后得到的是7*7，特征图是128个。  
* 全连接层把特征图转换为向量。输入的是7*7*128输出的是1024维向量。

In [3]:
n_input  = 784
n_output = 10
weights  = {
        # 3，3卷积核大小，1输入深度（灰度图深度为1），64：第一层卷积完后得到64个特征图
        'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.1)),
        # 输入深度64，输出深度128
        'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.1)),
        # 7*7*128
        'wd1': tf.Variable(tf.random_normal([7*7*128, 1024], stddev=0.1)),
        'wd2': tf.Variable(tf.random_normal([1024, n_output], stddev=0.1))
    }
biases   = {
        'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
        'bc2': tf.Variable(tf.random_normal([128], stddev=0.1)),
        'bd1': tf.Variable(tf.random_normal([1024], stddev=0.1)),
        'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1))
    }

stride=(b,h,w,c)参数的含义：  

b表示在样本上的步长默认为1，也就是每一个样本都会进行运算，就是不跳过任何一个样本  

h表示在高度上的默认移动步长为1，这个可以自己设定，根据网络的结构合理调节。  

w表示在宽度上的默认移动步长为1，这个同上可以自己设定。  

c表示在通道上的默认移动步长为1，这个表示每一个通道都会进行运算。

In [6]:
def conv_basic(_input, _w, _b, _keepratio):
        # INPUT 把输入格式转换为tensorflow格式：tesorflow支持四维格式
        # [n,h,w,c] n:batch大小，h图像高，w图像宽，c：深度（图像通道数)
        # -1:tensorflow自己推断
        _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
        # 卷积层1
        # strides:四维格式，1：batchsize上stride大小，1：在h上大小，1：在w上大小，1：通道上
        # padding：same（用0填充），valid(不填充)。区别：same:卷积时不够的0填充，valid：卷积时多余的给扔掉，一般用same
        _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
        # 可使用tf.nn.moments和tf.nn.batch_normalization进行标准化操作
        #_mean, _var = tf.nn.moments(_conv1, [0, 1, 2])
        #_conv1 = tf.nn.batch_normalization(_conv1, _mean, _var, 0, 1, 0.0001)
        _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1, _b['bc1']))
        # 选择2*2窗口，stride也设为2*2
        # ksize=[1, 2, 2, 1]，也是四个维度，1：batchsize上大小，2：在h上大小，2：在w上大小，1：通道上
        _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
        # 卷积层2
        _conv2 = tf.nn.conv2d(_pool_dr1, _w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
        #_mean, _var = tf.nn.moments(_conv2, [0, 1, 2])
        #_conv2 = tf.nn.batch_normalization(_conv2, _mean, _var, 0, 1, 0.0001)
        _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2, _b['bc2']))
        _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
    
        # _w['wd1'].get_shape().as_list()[0]拿到7*7*128=6272，改变_pool_dr2的shape，进入全连接层
        _dense1 = tf.reshape(_pool_dr2, [-1, _w['wd1'].get_shape().as_list()[0]])
        # 全连接层1
        _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
        _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
        # 全连接层2，输出结果
        _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
        # RETURN
        out = { 'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1, 'pool1_dr1': _pool_dr1,
            'conv2': _conv2, 'pool2': _pool2, 'pool_dr2': _pool_dr2, 'dense1': _dense1,
            'fc1': _fc1, 'fc_dr1': _fc_dr1, 'out': _out
        }
        return out
print ("CNN READY")

CNN READY


In [12]:
weights['wd1'].get_shape().as_list()

[6272, 1024]

In [14]:
#print (help(tf.nn.conv2d))
print (help(tf.nn.max_pool))

Help on function max_pool in module tensorflow.python.ops.nn_ops:

max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)
    Performs the max pooling on the input.
    
    Args:
      value: A 4-D `Tensor` of the format specified by `data_format`.
      ksize: A 1-D int Tensor of 4 elements.  The size of the window for
        each dimension of the input tensor.
      strides: A 1-D int Tensor of 4 elements.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. The padding algorithm.
        See the @{tf.nn.convolution$comment here}
      data_format: A string. 'NHWC', 'NCHW' and 'NCHW_VECT_C' are supported.
      name: Optional name for the operation.
    
    Returns:
      A `Tensor` of format specified by `data_format`.
      The max pooled output tensor.

None


In [7]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32) # 定义dropout率

# FUNCTIONS
# 定义损失函数和优化项

_pred = conv_basic(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) 
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) 
init = tf.global_variables_initializer()
    
# SAVER
print ("GRAPH READY")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

GRAPH READY


In [8]:
sess = tf.Session()
sess.run(init)

training_epochs = 100
batch_size      = 16
display_step    = 1
for epoch in range(training_epochs):
    avg_cost = 0.
    #total_batch = int(mnist.train.num_examples/batch_size)
    # 这里直接取10，为了运行更快一些
    total_batch = 10
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio:0.7})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})/total_batch

    # Display logs per epoch step
    if epoch % display_step == 0: 
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})
        print (" Training accuracy: %.3f" % (train_acc))
        #test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel, keepratio:1.})
        # print (" Test accuracy: %.3f" % (test_acc))

print ("OPTIMIZATION FINISHED")

Epoch: 000/100 cost: 6.843760490
 Training accuracy: 0.125
Epoch: 001/100 cost: 2.816543746
 Training accuracy: 0.562
Epoch: 002/100 cost: 1.483182317
 Training accuracy: 0.688
Epoch: 003/100 cost: 1.221699238
 Training accuracy: 0.438
Epoch: 004/100 cost: 1.114191836
 Training accuracy: 0.562
Epoch: 005/100 cost: 1.130992258
 Training accuracy: 0.875
Epoch: 006/100 cost: 0.970123154
 Training accuracy: 0.688
Epoch: 007/100 cost: 0.874241358
 Training accuracy: 0.750
Epoch: 008/100 cost: 0.846452421
 Training accuracy: 0.812
Epoch: 009/100 cost: 0.577680364
 Training accuracy: 0.938
Epoch: 010/100 cost: 0.679830244
 Training accuracy: 0.938
Epoch: 011/100 cost: 0.731648844
 Training accuracy: 0.875
Epoch: 012/100 cost: 0.642634910
 Training accuracy: 0.812
Epoch: 013/100 cost: 0.570261392
 Training accuracy: 0.562
Epoch: 014/100 cost: 0.630688891
 Training accuracy: 1.000
Epoch: 015/100 cost: 0.483516154
 Training accuracy: 0.938
Epoch: 016/100 cost: 0.374290504
 Training accuracy: 1.0